In [1]:
import numpy as np
import pandas as pd
import os
from tensorflow.keras.applications.vgg16 import preprocess_input
import tensorflow
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Flatten,Dropout
import os
import matplotlib.pyplot as plt

In [2]:
tumor_path='Brain Tumor Data Set/Brain Tumor/'
healthy_path='Brain Tumor Data Set/Healthy/'
filepaths = []
labels= []
dict_list = [tumor_path, healthy_path]
for i, j in enumerate(dict_list):
    flist=os.listdir(j)
    for f in flist:
        fpath=os.path.join(j,f)
        filepaths.append(fpath)
        if i==0:
            labels.append('Brain Tumor')
        else:
            labels.append('Healthy')
df=pd.DataFrame({'Filepath': filepaths,'label': labels})
df.head()

,Filepath,label
0,Brain Tumor Data Set/Brain Tumor/Cancer (1).jpg,Brain Tumor
1,Brain Tumor Data Set/Brain Tumor/Cancer (1).png,Brain Tumor
2,Brain Tumor Data Set/Brain Tumor/Cancer (1).tif,Brain Tumor
3,Brain Tumor Data Set/Brain Tumor/Cancer (10).jpg,Brain Tumor
4,Brain Tumor Data Set/Brain Tumor/Cancer (10).tif,Brain Tumor


In [3]:
train_df, val_df = train_test_split(df, test_size=0.3, random_state=42)

In [4]:
total_train = train_df.shape[0]
total_valid = val_df.shape[0]

In [5]:
train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input,
                                  rotation_range=15,
                                  shear_range=0.1,
                                  zoom_range=0.2,
                                  horizontal_flip=True,
                                  width_shift_range=0.1,
                                  height_shift_range=0.1)

train_generator = train_datagen.flow_from_dataframe(train_df,
                                                  x_col='Filepath',y_col='label',
                                                  target_size=(224,224),
                                                  batch_size=32,
                                                  class_mode="categorical")

valid_datagen = ImageDataGenerator(preprocessing_function= preprocess_input)
valid_generator = valid_datagen.flow_from_dataframe(val_df,
                                                   x_col='Filepath',y_col='label',
                                                   target_size=(224,224),
                                                   batch_size=32,
                                                   class_mode="categorical")

Found 3220 validated image filenames belonging to 2 classes.
Found 1380 validated image filenames belonging to 2 classes.


In [6]:
train_generator.class_indices

{'Brain Tumor': 0, 'Healthy': 1}

In [7]:
valid_generator.class_indices

{'Brain Tumor': 0, 'Healthy': 1}

In [8]:
vgg_model=tensorflow.keras.applications.vgg16.VGG16()

In [9]:
model=Sequential()
for layer in vgg_model.layers:
    model.add(layer)

In [10]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 56, 56, 256)       2

In [11]:
model.pop()

In [12]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 56, 56, 256)       2

In [13]:
for layer in model.layers:
    layer.trainable=False

In [14]:
model.add(Dense(2,activation='softmax'))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 56, 56, 256)       2

In [15]:
model.compile(optimizer=keras.optimizers.Adam(0.0001),loss='categorical_crossentropy',metrics=['accuracy'])

In [16]:
history= model.fit_generator(train_generator,
                   epochs=20,
                   validation_data=valid_generator,
                   validation_steps= total_valid//32,
                   steps_per_epoch= total_train//32)

C:\Users\ASUS\AppData\Local\Programs\Python\Python39\lib\site-packages\tensorflow\python\keras\engine\training.py:1940: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/20
100/100 [==============================] - 58s 490ms/step - loss: 0.6576 - accuracy: 0.6631 - val_loss: 0.4649 - val_accuracy: 0.7754
Epoch 2/20
100/100 [==============================] - 43s 431ms/step - loss: 0.3901 - accuracy: 0.8287 - val_loss: 0.3643 - val_accuracy: 0.8467
Epoch 3/20
100/100 [==============================] - 43s 433ms/step - loss: 0.3288 - accuracy: 0.8607 - val_loss: 0.3230 - val_accuracy: 0.8677
Epoch 4/20
100/100 [==============================] - 43s 431ms/step - loss: 0.2886 - accuracy: 0.8821 - val_loss: 0.2935 - val_accuracy: 0.8779
Epoch 5/20
100/100 [==============================] - 43s 432ms/step - loss: 0.2775 - accuracy: 0.8883 - val_loss: 0.2743 - val_accuracy: 0.8932
Epoch 6/20
100/100 [==============================] - 42s 416ms/step - loss: 0.2571 - accuracy: 0.8952 - val_loss: 0.2637 - val_accuracy: 0.8961
Epoch 7/20
100/100 [==============================] - 43s 432ms/step - loss: 0.2572 - accuracy: 0.8981 - val_loss: 0.2468 - val_ac

In [17]:
from tensorflow.keras.models import save_model
save_model(model, "tumor_detector.h5")